# General Observing

This notebook is used for general observing with the Auxiliary Telescope.

Craig Lage - 25-May-21

In [1]:
print(1)

1


In [2]:
import sys
import asyncio
import time
import os
import numpy as np

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [3]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [4]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [5]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.05 sec
Read 7 history items for RemoteEvent(ATSpectrograph, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATSpectrograph, 0, authList)
Read 100 history items for RemoteEvent(ATSpectrograph, 0, disperserInPosition)
Read 100 history items for RemoteEvent(ATSpectrograph, 0, filterInPosition)
Read 14 history items for RemoteEvent(ATSpectrograph, 0, fwState)
Read 23 history items for RemoteEvent(ATSpectrograph, 0, gwState)
Read 100 history items for RemoteEvent(ATSpectrograph, 0, heartbeat)
Read 1 history items for RemoteEvent(ATSpectrograph, 0, logLevel)
Read 3 history items for Re

[[None, None, None, None, None, None, None], [None, None, None, None]]

In [8]:
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [9]:
# enable components
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
#await latiss.enable({"atspectrograph": "current"})

Enabling all components
Gathering settings.
Received settings from users.: {'atdome': 'current', 'ataos': 'current', 'athexapod': 'current'}
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for atpneumatics.
Complete settings for atdometrajectory.
Settings versions: {'atdome': 'current', 'ataos': 'current', 'athexapod': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'atdometrajectory': ''}
[atmcs]::[<State.ENABLED: 2>]
[atptg]::[<State.ENABLED: 2>]
[ataos]::[<State.ENABLED: 2>]
[atpneumatics]::[<State.ENABLED: 2>]
[athexapod]::[<State.ENABLED: 2>]
[atdome]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atdometrajectory]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [6]:
await latiss.enable({"atspectrograph": "current"})

Enabling all components
Gathering settings.
Received settings from users.: {'atspectrograph': 'current'}
Couldn't get settingVersions event. Using empty settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atcamera.
Complete settings for atheaderservice.
Complete settings for atarchiver.
Settings versions: {'atspectrograph': 'current', 'atcamera': '', 'atheaderservice': '', 'atarchiver': ''}
[atcamera]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atspectrograph]::[<State.ENABLED: 2>]
[atheaderservice]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atarchiver]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [7]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021070800093])

In [ ]:
# Take 50 biases seq # 002-051
# Added wait to stop killing the recent images
for i in range(50):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
# Take 10 10 second darks 52-61
await latiss.take_darks(10.0, 10)

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.ENABLED, settingsToApply='current')

In [16]:
await salobj.set_summary_state(atcs.rem.ataos, salobj.State.ENABLED, settingsToApply='current')

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [26]:
await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [39]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [40]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

logMessage DDS read queue is filling: 18 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements


In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
# Take a test flat
await latiss.take_flats(2.0, 1, filter='empty_1', grating='empty_1')

In [ ]:
# Take 10 2 second flats 72-81
await latiss.take_flats(2.0, 10, filter='empty_1', grating='empty_1')

In [ ]:
# Take 10 2 second flats 72-81
await latiss.take_flats(2.0, 10, filter='RG610', grating='empty_1')

In [ ]:
# Take 10 2 second flats 72-81
await latiss.take_flats(2.0, 10, filter='BG40', grating='empty_1')

In [ ]:
# Take flats for PTC 82-141
# Added wait to stop killing the recent images
for i in range(30):
    exp = 0.2 * float(i+1)
    await latiss.take_flats(exp, 1, filter='RG610', grating='empty_1')
    if exp < 2.0:
        await asyncio.sleep(2.0)
    await latiss.take_flats(exp, 1, filter='RG610', grating='empty_1')


In [ ]:
# This moves everything to park position and opens the dome.  It takes 5+ minutes.
await atcs.prepare_for_onsky()

In [ ]:
await atcs.close_m1_cover()

In [ ]:
await atcs.close_m1_vent()

In [27]:
await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.set_start(timeout=5.0)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=433798456, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='ERROR: Command masterCloseValve rejected while in EnabledState state.')

In [ ]:
await atcs.rem.atpneumatics.cmd_open

In [28]:
await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.set_start(timeout=5.0)

In [29]:
await atcs.rem.atpneumatics.cmd_m1CloseAirValve.start(timeout=5.0)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=858386508, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='ERROR: Command m1CloseValve rejected while in EnabledState state.')

In [30]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(
    m1=True, hexapod=True, atspectrograph=True, timeout=atcs.long_timeout
) 

In [31]:
await atcs.open_m1_cover()

Cover state <MirrorCoverState.CLOSED: 6>
Opening M1 cover.
Cover state <MirrorCoverState.CLOSED: 6>
Cover state <MirrorCoverState.INMOTION: 8>
Cover state <MirrorCoverState.OPENED: 7>


In [32]:
await atcs.open_m1_vent()

M1 vent state <VentsPosition.CLOSED: 1>
Opening M1 vents.
M1 vent state <VentsPosition.PARTIALLYOPENED: 2>
M1 vent state <VentsPosition.OPENED: 0>
logMessage DDS read queue is filling: 13 of 100 elements
logMessage DDS read queue is filling: 85 of 100 elements


In [ ]:
# Slew to a given object and start tracking.
await atcs.slew_object('HD117637', rot_type=RotType.PhysicalSky)

In [11]:
await atcs.rem.atptg.cmd_pointLoadModel.set_start(pointingFile="/home/saluser/at_20210610_fauto.mod")

In [ ]:
await atcs.disable_dome_following()

In [ ]:
await atcs.slew_dome_to(45.0)

In [ ]:
await atcs.home_dome()

In [10]:
await atcs.enable_dome_following()

Enable dome trajectory following.


In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis="z")

In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis="y")

In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis="x")

In [ ]:
await atcs.rem.ataos.cmd_offset.set_start(z=-0.4)

In [ ]:
await atcs.offset_xy(y=-110, x=-150, absorb=False)

In [ ]:
# Take 1 5 second image
await latiss.take_object(5.0, 1, filter='RG610', grating='empty_1')

In [ ]:
for i in range(20):
    await latiss.take_object(5.0, 1, filter='RG610', grating='empty_1')
    await asyncio.sleep(30.0)
    

In [ ]:
await latiss.take_engtest(5.0, 1, filter='RG610', grating='empty_1')

In [ ]:
await latiss.setup_atspec(filter='RG610', grating='empty_1')

In [ ]:
await latiss.take_object(10.0, 1, filter='empty_1', grating='ronchi90lpmm')

In [ ]:
await atcs.rem.atpneumatics.cmd_closeM1Cover.start()

In [ ]:
await atcs.home_dome()

In [ ]:
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [ ]:
await atcs.slew_dome_to(az=90.0)

In [ ]:
await atcs.open_dome_shutter()

In [ ]:
await atcs.rem.atpneumatics.cmd_openM1CellVents.start()

In [ ]:
# This Opens/Closes the dropout shutter
# await atcs.rem.atdome.cmd_moveShutterDropoutDoor.set_start(open=False)

In [23]:
# This tunrs on ATAOS corrections and turns on the air pressure under the M1 mirror.
await atcs.rem.ataos.cmd_enableCorrection.set_start(
    m1=True, hexapod=True, atspectrograph=True, timeout=atcs.long_timeout
)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=730611388, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: msg='Command failed', ackcmd=(ackcmd private_seqNum=1668546129, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='ERROR: Command masterOpenValve rejected while in EnabledState state.')")

In [ ]:
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})

In [ ]:
await atcs.rem.atdometrajectory.cmd_setFollowingMode.set_start(enable=True)

In [ ]:
# Pointing to a given Az/El  This will not track
await atcs.point_azel(az, el, rot_tel=rot)

In [ ]:
# Pointing to a given RA/Dec  This will track
await atcs.slew_icrs(ra=ra, dec=dec, rot=rot, rot_type=RotType.PhysicalSky)

In [ ]:
# Slew to a given object and start tracking.
await atcs.slew_object('HD 187101', rot_type=RotType.Parallactic)

In [ ]:
# It is recommended to do this and wait several seconds for the spectrograph to settle
# before taking an image
await latiss.setup_atspec(filter='RG610', grating='empty_1')

In [ ]:
# Take 1 2 second image
await latiss.take_object(30.0, 1, filter='RG610', grating='empty_1')

In [ ]:
await atcs.rem.ataos.cmd_disableCorrection.set_start(atspectrograph=True)

In [ ]:
# This loop will take a set of images on both sides of current best focus
# starts at starting_z_offset and runs to -starting_z_offset
# then puts the z_offset back where it was.

starting_z_offset = -0.20
z_offset_increment = 0.05
nsteps = int((-2 * starting_z_offset) / z_offset_increment) + 1
total_z_offset = 0.0
await atcs.rem.ataos.cmd_offset.set_start(z=starting_z_offset)
total_z_offset += starting_z_offset
print(f"Total z offset = {total_z_offset}")
await asyncio.sleep(2)
for i in range(nsteps):
    await latiss.take_object(5.0, 1, filter='RG610', grating='holo4_003')
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_increment)
    total_z_offset += z_offset_increment
    print(f"Total z offset = {total_z_offset}")
    
# Put offset back where it was
await atcs.rem.ataos.cmd_offset.set_start(z=-total_z_offset)
total_z_offset -= total_z_offset
print(f"Total z offset = {total_z_offset}")
    

In [ ]:
# This will put in a hexapod offset
#await atcs.rem.ataos.cmd_offset.set_start(z=-0.095)

In [ ]:
# To reset all hexapod offsets
tmp = await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
# Move the star within the field
# Offsets are in arcseconds.
#await atcs.offset_xy(y=-100, x=0, relative=True)

In [ ]:
await latiss.take_object(5.0, 1, filter='RG610', grating='ronchi170lpmm')

In [ ]:
atcs.offset_azel?

In [ ]:
# Run a figure-8 of offsets
offsets = [[0,0], [0,100], [100,0], [0,-100], [-100,0], [0,-100], [-100,0], [0,100], [100,0]]
await latiss.setup_atspec(filter='RG610', grating='empty_1')
await asyncio.sleep(2)
for [xx, yy] in offsets:
    await atcs.offset_azel(az=xx, el=yy, relative=True)
    await asyncio.sleep(2)
    await latiss.take_object(2.0, 1, filter='RG610', grating='empty_1')
    await asyncio.sleep(2)
    

In [ ]:
# To enable or disable built-in offsets for the filters and gratings
#await atcs.rem.ataos.cmd_disableCorrection.set_start(atspectrograph=True)
#await atcs.rem.ataos.cmd_enableCorrection.set_start(atspectrograph=True)

In [ ]:
# To stop tracking
# await atcs.stop_tracking()

In [ ]:
# If you are done, put things in standby, or shut things down completely
#await atcs.standby()
await atcs.shutdown()

In [41]:
# Putting everything back in standby.
await latiss.standby()

[atcamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atspectrograph]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [36]:
await atcs.rem.atdome.cmd_moveShutterDropoutDoor.set_start(open=False)

logMessage DDS read queue is filling: 50 of 100 elements
logMessage DDS read queue is filling: 74 of 100 elements
